# CS202: Compiler Construction

## In-class Exercises, Week of 02/06/2023

----

# Register allocation

*Register allocation* is the process of assigning register locations to program variables.

## Question 1

Why is register allocation important?

Register allocation is maybe the biggest optimization there is. Registers are much faster than memory. If you can keep your variables in registers, you can make your program run much faster.

## Question 2

Consider the program:

```
x = 5
y = x + 6
y + 7
```

Here is the equivalent in x86 assembly, using the stack to hold variable values.

```
...
start:
  movq $5, -8(%rbp)
  movq -8(%rbp), %rax
  movq %rax, -16(%rbp)
  addq $6, -16(%rbp)
  movq -16(%rbp), %rax
  movq %rax, -24(%rbp)
  addq $7, -24(%rbp)
  movq -24(%rbp), %rax
  jmp conclusion
...
```

Convert this program into one that only uses registers.

Let's assign each stack location to a register instead, ad find-and-replace.

- -8 -> r8
- -16 -> r9
- -24 -> r10

```
  movq $5, %r8
  movq %r8, %rax
  movq %rax, %r9
  addq $6, %r9
  movq %r9, %rax
  movq %rax,%r10
  addq $7,%r10
  movq%r10, %rax
  jmp conclusion
```

## Question 3

How many registers are *actually needed* to represent this program?

For this exercise we actually needed only 1 register because all the variables can share one location without interfering with each other.

## Question 4

Why is *optimal* register allocation hard?

Optimal register allocation is NP-complete.

We need an approximately optimal soution that runs in a reasonable amount of time (quadratic time is fine)

## Note

Our solution to register allocation will involve three parts:

1. [*Liveness analysis*](https://en.wikipedia.org/wiki/Live_variable_analysis), to understand when a variable's value is no longer needed
2. Building an *interference graph* with edges between variables that cannot share a register
3. Allocating registers to variables by *coloring the interference graph* ([graph coloring](https://en.wikipedia.org/wiki/Graph_coloring) is NP-hard)

----

# Liveness Analysis

The first part of the allocate-registers pass performs a *liveness analysis* on the program.

## Question 5

Here is the pseudo-x86 equivalent of the program from earlier, written in x86 assembly syntax:

```
start:
 movq $5, x
 movq x, y
 addq $6, y
 movq y, r
 addq $7, r
 movq r, %rax
```

In the table below, annotate each instruction with its *live-after set*.

Use the following rule to determine the live-after set for instruction $k$:

\begin{align}
L_{after}(k) = (L_{after}(k+1) − W (k+1)) \cup R(k+1)
\end{align}

where $W$ means the variables written by the instruction, and $R$ means the variables read by an instruction. See section 3.2 in the textbook for more.

**Solution**:

| Instruction     | Live-After Set |
| :-------------- | :------------- |
| `start:`        |    |
| `movq $5, x`    |  {x}  |
| `movq x, y`     |  {y}  |
| `addq $6, y`    |  {y}  |
| `movq y, r`     |  {r}  |
| `addq $7, r`    |  {r}  |
| `movq r, %rax`  |  {}  |

## Question 6

Describe the liveness analysis in the compiler.

- Proces the instructions in reverse order
- Start with empty live-after set
- Produce the next live-after set using the previous one based on the rule above

----
- `ul_instr` given a instuction k+1, and the live-after set for instruction k+1, produce the live-after set for instruction k
- `ul_block`:
  - start with empty list of live-after sets
  - start with empty current live-after set
  - loop over instructions in reverse order
    - call `ul_instr` to get the next live-after set
    - add it to the list
  - at the end, reverse the list of live-after sets and return it

----

# Interference Graph

The second part of the allocate-registers pass will build an *interference graph* using the live-after sets computed earlier. The interference graph is an undirected graph with a node for each program variable, and an edge between each pair of variables which may *not* share a home.

## Question 7

Using the instructions and live-after sets from question 5, draw the interference graph.

Use the following principle: *the writes performed by an instruction must not overwrite something in a live location*.

The interference graph has no edges, hence all the variables can share the same register.

## Question 8

Using the instructions and live-after sets below, draw the interference graph.

| Instruction    | Live-After Set |
| :------------- | :------------- |
| `start:`       | {} |
| `movq $1, a`   | {a}|
| `movq $2, b`   | {a, b} |
| `movq $3, c`   | {a, b, c} |
| `movq a, t`    | {b, c, t} |
| `addq b, t`    | {c, t} |
| `movq t, r`    | {c, r} |
| `addq c, r`    | {r} |
| `movq r, %rax` | {}  |


Edges are:
- a - b
- b - c
- b - t
- c - t
- c - r
- a - c

## Question 9

What variables in the above graph can share a register? What variables can't?

- A and C can share a register
- B and T can share a register
- R cannot share a register with any other variable
- B and C cannot share a register